## Recommendations with MovieTweetings: Collaborative Filtering

One of the most popular methods for making recommendations is **collaborative filtering**.  In collaborative filtering, you are using the collaboration of user-item recommendations to assist in making new recommendations.  

There are two main methods of performing collaborative filtering:

1. **Neighborhood-Based Collaborative Filtering**, which is based on the idea that we can either correlate items that are similar to provide recommendations or we can correlate users to one another to provide recommendations.

2. **Model Based Collaborative Filtering**, which is based on the idea that we can use machine learning and other mathematical models to understand the relationships that exist amongst items and users to predict ratings and provide ratings.


In this notebook, you will be working on performing **neighborhood-based collaborative filtering**.  There are two main methods for performing collaborative filtering:

1. **User-based collaborative filtering:** In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

2. **Item-based collaborative filtering:** In this type of recommendation, first you need to find the items that are most related to each other item (based on similar ratings).  Then you can use the ratings of an individual on those similar items to understand if a user will like the new item.

In this notebook you will be implementing **user-based collaborative filtering**.  However, it is easy to extend this approach to make recommendations using **item-based collaborative filtering**.  First, let's read in our data and necessary libraries.

**NOTE**: Because of the size of the datasets, some of your code cells here will take a while to execute, so be patient!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from scipy.sparse import csr_matrix
from IPython.display import HTML
from progressbar import ProgressBar

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

print(movies[:5])
print()
print(reviews[:5])

   movie_id                                              movie  \
0         8      Edison Kinetoscopic Record of a Sneeze (1894)   
1        10                La sortie des usines Lumière (1895)   
2        12                      The Arrival of a Train (1896)   
3        25  The Oxford and Cambridge University Boat Race ...   
4        91                         Le manoir du diable (1896)   

               genre  date  1800's  1900's  2000's  History  News  Horror  \
0  Documentary|Short  1894       1       0       0        0     0       0   
1  Documentary|Short  1895       1       0       0        0     0       0   
2  Documentary|Short  1896       1       0       0        0     0       0   
3                NaN  1895       1       0       0        0     0       0   
4       Short|Horror  1896       1       0       0        0     0       1   

     ...     Fantasy  Romance  Game-Show  Action  Documentary  Animation  \
0    ...           0        0          0       0            1   

### Measures of Similarity

When using **neighborhood** based collaborative filtering, it is important to understand how to measure the similarity of users or items to one another.  

There are a number of ways in which we might measure the similarity between two vectors (which might be two users or two items).  In this notebook, we will look specifically at two measures used to compare vectors:

* **Pearson's correlation coefficient**

Pearson's correlation coefficient is a measure of the strength and direction of a linear relationship. The value for this coefficient is a value between -1 and 1 where -1 indicates a strong, negative linear relationship and 1 indicates a strong, positive linear relationship. 

If we have two vectors x and y, we can define the correlation between the vectors as:


$$CORR(x, y) = \frac{\text{COV}(x, y)}{\text{STDEV}(x)\text{ }\text{STDEV}(y)}$$

where 

$$\text{STDEV}(x) = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}$$

and 

$$\text{COV}(x, y) = \frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

where n is the length of the vector, which must be the same for both x and y and $\bar{x}$ is the mean of the observations in the vector.  

We can use the correlation coefficient to indicate how alike two vectors are to one another, where the closer to 1 the coefficient, the more alike the vectors are to one another.  There are some potential downsides to using this metric as a measure of similarity.  You will see some of these throughout this workbook.


* **Euclidean distance**

Euclidean distance is a measure of the straightline distance from one vector to another.  Because this is a measure of distance, larger values are an indication that two vectors are different from one another (which is different than Pearson's correlation coefficient).

Specifically, the euclidean distance between two vectors x and y is measured as:

$$ \text{EUCL}(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

Different from the correlation coefficient, no scaling is performed in the denominator.  Therefore, you need to make sure all of your data are on the same scale when using this metric.

**Note:** Because measuring similarity is often based on looking at the distance between vectors, it is important in these cases to scale your data or to have all data be in the same scale.  If some measures are on a 5 point scale, while others are on a 100 point scale, you are likely to have non-optimal results due to the difference in variability of your features.  In this case, we will not need to scale data because they are all on a 10 point scale, but it is always something to keep in mind!

------------

### User-Item Matrix

In order to calculate the similarities, it is common to put values in a matrix.  In this matrix, users are identified by each row, and items are represented by columns.  


![alt text](images/userxitem.png "User Item Matrix")


In the above matrix, you can see that **User 1** and **User 2** both used **Item 1**, and **User 2**, **User 3**, and **User 4** all used **Item 2**.  However, there are also a large number of missing values in the matrix for users who haven't used a particular item.  A matrix with many missing values (like the one above) is considered **sparse**.

Our first goal for this notebook is to create the above matrix with the **reviews** dataset.  However, instead of 1 values in each cell, you should have the actual rating.  

The users will indicate the rows, and the movies will exist across the columns. To create the user-item matrix, we only need the first three columns of the **reviews** dataframe, which you can see by running the cell below.

In [2]:
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_items.head()

,user_id,movie_id,rating
0,1,68646,10
1,1,113277,10
2,2,422720,8
3,2,454876,8
4,2,790636,7


### Creating the User-Item Matrix

In order to create the user-items matrix (like the one above), I personally started by using a [pivot table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html). 

However, I quickly ran into a memory error (a common theme throughout this notebook).  I will help you navigate around many of the errors I had, and achieve useful collaborative filtering results! 

_____

`1.` Create a matrix where the users are the rows, the movies are the columns, and the ratings exist in each cell, or a NaN exists in cells where a user hasn't rated a particular movie. If you get a memory error (like I did), [this link here](https://stackoverflow.com/questions/39648991/pandas-dataframe-pivot-memory-error) might help you!

In [3]:
# Create user-by-item matrix
#user_by_movie = pd.pivot_table(user_items, values='rating', index='user_id', columns='movie_id')
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
user_by_movie.head()

movie_id,8,10,12,25,91,417,439,443,628,833,...,8144778,8144868,8206708,8289196,8324578,8335880,8342748,8342946,8402090,8439854
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
user_1_movies = user_by_movie.loc[1][user_by_movie.loc[1].isnull() == False]
print('user_1_movies = \n{}\n'.format(user_1_movies))
user_1_movies_index = user_1_movies.index
print('user_1_movies_index = \n{}\n'.format(user_1_movies_index))
user_1_movies_index.values

user_1_movies = 
movie_id
68646     10.0
113277    10.0
Name: 1, dtype: float64

user_1_movies_index = 
Int64Index([68646, 113277], dtype='int64', name='movie_id')



array([ 68646, 113277])

Check your results below to make sure your matrix is ready for the upcoming sections.

In [5]:
assert movies.shape[0] == user_by_movie.shape[1], \
       "Oh no! Your matrix should have {} columns, and yours has {}!".format(
           movies.shape[0], user_by_movie.shape[1])
assert reviews.user_id.nunique() == user_by_movie.shape[0], \
       "Oh no! Your matrix should have {} rows, and yours has {}!".format(
           reviews.user_id.nunique(), user_by_movie.shape[0])
print("Looks like you are all set! Proceed!")
HTML('<img src="images/greatjob.webp">')

Looks like you are all set! Proceed!


`2.` Now that you have a matrix of users by movies, use this matrix to create a dictionary where the key is each user and the value is an array of the movies each user has rated.

In [6]:
# Create a dictionary with users and corresponding movies seen

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values

    return movies


def create_user_movie_dict():
    '''
    INPUT: None
    OUTPUT: movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    Creates the movies_seen dictionary
    '''
    num_users = user_by_movie.shape[0]
    movies_seen = dict()
    
    for id in range(1, num_users + 1):
        movies_seen[id] = movies_watched(id)
    
    return movies_seen


# Use your function to return dictionary
movies_seen = create_user_movie_dict()
dict(list(movies_seen.items())[:5])    # similar to DataFrame.head()

{1: array([ 68646, 113277]),
 2: array([ 422720,  454876,  790636,  816711, 1091191, 1103275, 1322269,
        1390411, 1398426, 1431045, 1433811, 1454468, 1535109, 1675434,
        1798709, 2017038, 2024544, 2294629, 2361509, 2381249, 2726560,
        2883512, 3079380]),
 3: array([1790864, 2170439, 2203939]),
 4: array([1300854]),
 5: array([ 54953, 120863])}

`3.` If a user hasn't rated more than 2 movies, we consider these users "too new".  Create a new dictionary that only contains users who have rated more than 2 movies.  This dictionary will be used for all the final steps of this workbook.

In [7]:
# Remove individuals who have watched 2 or fewer movies - don't have enough data to make recs

def create_movies_to_analyze(movies_seen, lower_bound=2):
    '''
    INPUT:  
    movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    lower_bound - (an int) a user must have more movies seen than the lower bound
                           to be added to the movies_to_analyze dictionary

    OUTPUT: 
    movies_to_analyze - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    The movies_seen and movies_to_analyze dictionaries should be the same except that the output dictionary has removed 
    
    '''
    movies_to_analyze = dict()
    
    for usr, mov_list in movies_seen.items():
        if len(mov_list) > lower_bound:
            movies_to_analyze[usr] = mov_list
    
    return movies_to_analyze


# Use your function to return your updated dictionary
movies_to_analyze = create_movies_to_analyze(movies_seen)
dict(list(movies_to_analyze.items())[:5])

{2: array([ 422720,  454876,  790636,  816711, 1091191, 1103275, 1322269,
        1390411, 1398426, 1431045, 1433811, 1454468, 1535109, 1675434,
        1798709, 2017038, 2024544, 2294629, 2361509, 2381249, 2726560,
        2883512, 3079380]),
 3: array([1790864, 2170439, 2203939]),
 7: array([1764234, 1790885, 2053463]),
 8: array([ 385002, 1220198, 1462900, 1512685, 1631707, 1986994, 1999995]),
 9: array([ 65207, 363163, 985699])}

In [8]:
# Run the tests below to check that your movies_to_analyze matches the solution
assert len(movies_to_analyze) == 23512, "Oops!  It doesn't look like your dictionary has the right number of individuals."
assert len(movies_to_analyze[2]) == 23, "Oops!  User 2 didn't match the number of movies we thought they would have."
assert len(movies_to_analyze[7])  == 3, "Oops!  User 7 didn't match the number of movies we thought they would have."
print("If this is all you see, you are good to go!")

If this is all you see, you are good to go!


### Calculating User Similarities

Now that you have set up the **movies_to_analyze** dictionary, it is time to take a closer look at the similarities between users.  Below is the pseudocode for how I thought about determining the similarity between users:

```
for user1 in movies_to_analyze
    for user2 in movies_to_analyze
        see how many movies match between the two users
        if more than two movies in common
            pull the overlapping movies
            compute the distance/similarity metric between ratings on the same movies for the two users
            store the users and the distance metric
```

However, this took a very long time to run, and other methods of performing these operations did not fit on the workspace memory!

Therefore, rather than creating a dataframe with all possible pairings of users in our data, your task for this question is to look at a few specific examples of the correlation between ratings given by two users.  For this question consider you want to compute the [correlation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) between users.

`4.` Using the **movies_to_analyze** dictionary and **user_by_movie** dataframe, create a function that computes the correlation between the ratings of similar movies for two users.  Then use your function to compare your results to ours using the tests below.  

In [9]:
# Just playing around, figuring out how to extract the data...

movies_2 = list(movies_to_analyze.get(2))
movies_66 = list(movies_to_analyze.get(66))
print('movies_2 ({}) =\n{}\n\nmovies_66 ({}) = \n{}\n'.format(
    len(movies_2), movies_2[:10], len(movies_66), movies_66[:10]))
movies_2_66 = list(np.intersect1d(movies_2, movies_66))
print('movies_2_66 ({}) =\n{}\n'.format(len(movies_2_66), movies_2_66[:10]))

movies_2 (23) =
[422720, 454876, 790636, 816711, 1091191, 1103275, 1322269, 1390411, 1398426, 1431045]

movies_66 (267) = 
[33467, 33870, 36775, 37008, 37017, 38057, 38854, 39757, 40202, 40897]

movies_2_66 (3) =
[1454468, 1798709, 2883512]



In [10]:
users_2_66_by_movie = user_by_movie.loc[[2, 66]][movies_2_66]
users_2_66_by_movie.transpose()

user_id,2,66
movie_id,,
1454468,8.0,6.0
1798709,10.0,9.0
2883512,8.0,8.0


In [11]:
users_2_66_by_movie.transpose().corr()

user_id,2,66
user_id,,
2,1.000000,0.755929
66,0.755929,1.000000


In [12]:
def compute_correlation(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    movies_1_and_2 = list(np.intersect1d(movies_to_analyze[user1], movies_to_analyze[user2]))
    movies_1_and_2_df = user_by_movie.loc[[user1, user2], movies_1_and_2]
    
    corr = movies_1_and_2_df.transpose().corr().iloc[0, 1]
  
    return corr #return the correlation

In [13]:
# Test your function against the solution

corr_2_2 = compute_correlation(2,2)
print('corr_2_2 = ', corr_2_2)
assert corr_2_2 == 1.0, "Oops!  The correlation between a user and itself should be 1.0."

corr_2_66 = round(compute_correlation(2,66), 2)
print('corr_2_66 = ', corr_2_66)
assert corr_2_66 == 0.76, "Oops!  The correlation between user 2 and 66 should be about 0.76."

corr_2_104 = compute_correlation(2,104)
print('corr_2_104 = ', corr_2_104)
assert np.isnan(corr_2_104), "Oops!  The correlation between user 2 and 104 should be a NaN."

print("If this is all you see, then it looks like your function passed all of our tests!")

corr_2_2 =  1.0
corr_2_66 =  0.76
corr_2_104 =  nan
If this is all you see, then it looks like your function passed all of our tests!


### Why the NaN's?

If the function you wrote passed all of the tests, then you have correctly set up your function to calculate the correlation between any two users.  

`5.` But one question is, why are we still obtaining **NaN** values?  As you can see in the code cell above, users 2 and 104 have a correlation of **NaN**. Why?

Think and write your ideas here about why these NaNs exist, and use the cells below to do some coding to validate your thoughts. You can check other pairs of users and see that there are actually many NaNs in our data - 2,526,710 of them in fact. These NaN's ultimately make the correlation coefficient a less than optimal measure of similarity between two users.




In [14]:
# Which movies did both user 2 and user 104 see?
movies_2_104 = list(np.intersect1d(movies_to_analyze[2], movies_to_analyze[104]))
movies_2_104

[454876, 816711, 1454468, 1535109]

In [15]:
# What were the ratings for each user for those movies?
user_by_movie.loc[[2, 104], movies_2_104]

movie_id,454876,816711,1454468,1535109
user_id,,,,
2,8.0,8.0,8.0,8.0
104,9.0,7.0,7.0,9.0


`6.` Because the correlation coefficient proved to be less than optimal for relating user ratings to one another, we could instead calculate the euclidean distance between the ratings.  I found [this post](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) particularly helpful when I was setting up my function.  This function should be very similar to your previous function.  When you feel confident with your function, test it against our results.

In [16]:
# Just playing around, to better visualize the data

mov_2 = user_by_movie.loc[2, movies_2_66]
print('mov_2 =\n\n{}\n'.format(mov_2))
mov_66 = user_by_movie.loc[66, movies_2_66]
print('mov_66 =\n\n{}\n'.format(mov_66))
euc_dist_2_66 = np.linalg.norm(mov_2 - mov_66)
print('euc_dist_2_66 = ', round(euc_dist_2_66, 2))

mov_2 =

movie_id
1454468     8.0
1798709    10.0
2883512     8.0
Name: 2, dtype: float64

mov_66 =

movie_id
1454468    6.0
1798709    9.0
2883512    8.0
Name: 66, dtype: float64



/opt/conda/lib/python3.6/site-packages/numexpr/cpuinfo.py:42: UserWarning: [Errno 12] Cannot allocate memory
  warnings.warn(str(e), UserWarning, stacklevel=stacklevel)


euc_dist_2_66 =  2.24


In [17]:
def compute_euclidean_dist(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the euclidean distance between user1 and user2
    '''
    movies_1_and_2 = list(np.intersect1d(movies_to_analyze[user1], movies_to_analyze[user2]))
    movies_1_df = user_by_movie.loc[user1, movies_1_and_2]
    movies_2_df = user_by_movie.loc[user2, movies_1_and_2]
    
    dist = np.linalg.norm(movies_1_df - movies_2_df)
   
    return dist #return the euclidean distance

In [18]:
# Read in solution euclidean distances
import pickle
df_dists = pd.read_pickle("data/Term2/recommendations/lesson1/data/dists.p")
df_dists.head()

,user1,user2,eucl_dist
0,2,2,0.000000
1,2,66,2.236068
2,2,90,5.385165
3,2,99,2.828427
4,2,104,2.000000


In [19]:
# Test your function against the solution

euc_dist_2_2 = compute_euclidean_dist(2,2)
print('euc_dist_2_2 = {}\n'.format(euc_dist_2_2))
assert euc_dist_2_2 == df_dists.query("user1 == 2 and user2 == 2")['eucl_dist'][0], \
       "Oops! The distance between a user and itself should be 0.0."

euc_dist_2_66 = round(compute_euclidean_dist(2,66), 2)
print('euc_dist_2_66 = {}\n'.format(euc_dist_2_66))
assert euc_dist_2_66 == round(df_dists.query("user1 == 2 and user2 == 66")['eucl_dist'][1], 2), \
       "Oops!  The distance between user 2 and 66 should be about 2.24."

euc_dist_2_104 = compute_euclidean_dist(2,104)
print('euc_dist_2_104 = {}\n'.format(euc_dist_2_104))
assert np.isnan(euc_dist_2_104) == np.isnan(df_dists.query("user1 == 2 and user2 == 104")['eucl_dist'][4]), \
       "Oops!  The distance between user 2 and 104 should be 2."

print("If this is all you see, then it looks like your function passed all of our tests!")

euc_dist_2_2 = 0.0

euc_dist_2_66 = 2.24

euc_dist_2_104 = 2.0

If this is all you see, then it looks like your function passed all of our tests!


### Using the Nearest Neighbors to Make Recommendations

In the previous question, you read in **df_dists**. Therefore, you have a measure of distance between each user and every other user. This dataframe holds every possible pairing of users, as well as the corresponding euclidean distance.

Because of the **NaN** values that exist within the correlations of the matching ratings for many pairs of users, as we discussed above, we will proceed using **df_dists**. You will want to find the users that are 'nearest' each user.  Then you will want to find the movies the closest neighbors have liked to recommend to each user.

I made use of the following objects:

* df_dists (to obtain the neighbors)
* user_items (to obtain the movies the neighbors and users have rated)
* movies (to obtain the names of the movies)

`7.` Complete the functions below, which allow you to find the recommendations for any user.  There are five functions which you will need:

* **find_closest_neighbors** - this returns a list of user_ids from closest neighbor to farthest neighbor using euclidean distance


* **movies_liked** - returns an array of movie_ids


* **movie_names** - takes the output of movies_liked and returns a list of movie names associated with the movie_ids


* **make_recommendations** - takes a user id and goes through closest neighbors to return a list of movie names as recommendations


* **all_recommendations** = loops through every user and returns a dictionary of with the key as a user_id and the value as a list of movie recommendations

In [20]:
# Just playing around...

distances_66 = df_dists[(df_dists['user1'] == 66) & (df_dists['user2'] != 66)][['user2', 'eucl_dist']]
distances_66.sort_values('eucl_dist', ascending=True, inplace=True)
print('5 closest to 66:\n\n{}\n'.format(distances_66[:5]))
print('5 furthest from 66:\n\n{}'.format(distances_66[-6:-1]))

5 closest to 66:

       user2  eucl_dist
34506  33854        0.0
34972  37869        0.0
31927  12075        0.0
32076  13459        0.0
31627   9466        0.0

5 furthest from 66:

       user2  eucl_dist
31739  10481  17.000000
30902   3425  17.146428
33576  26173  17.175564
33067  21695  19.974984
36883  53966  20.174241


In [21]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # I treated ties as arbitrary and just kept whichever was easiest to keep using the head method
    # You might choose to do something less hand wavy - order the neighbors
    closest_neighbors = df_dists[(df_dists['user1'] == user) & (df_dists['user2'] != user)][['user2', 'eucl_dist']]
    closest_neighbors.sort_values(by='eucl_dist', ascending=True, inplace=True)
    
    return closest_neighbors['user2']

In [22]:
# ... and checking ...

find_closest_neighbors(66).head()

34506    33854
34972    37869
31927    12075
32076    13459
31627     9466
Name: user2, dtype: int64

In [23]:
# ... and playing ...

movies_66_7 = reviews[(reviews['user_id'] == 66) & (reviews['rating'] >= 7)][['movie_id', 'rating']]
movies_66_7.head()

,movie_id,rating
559,33467,8
560,33870,9
561,36775,9
562,37008,7
563,37017,7


In [24]:
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
    movies_liked - an array of movies the user has watched and liked
    '''
    movies_liked = reviews[(reviews['user_id'] == user_id) & (reviews['rating'] >= min_rating)]['movie_id']
    # The solution gives:
    # movies_liked = np.array(user_items.query('user_id == @user_id and rating > (@min_rating -1)')['movie_id'])
  
    return movies_liked

In [25]:
# ... and checking ...

movies_liked(66, min_rating=5).head()

559    33467
560    33870
561    36775
562    37008
563    37017
Name: movie_id, dtype: int64

In [26]:
# ... and playing ...

movies[movies['movie_id'].isin([8, 10, 12, 25, 91])]['movie']

0        Edison Kinetoscopic Record of a Sneeze (1894)
1                  La sortie des usines Lumière (1895)
2                        The Arrival of a Train (1896)
3    The Oxford and Cambridge University Boat Race ...
4                           Le manoir du diable (1896)
Name: movie, dtype: object

In [27]:
def movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = movies[movies['movie_id'].isin(movie_ids)]['movie']

    return movie_lst

In [28]:
# ... and checking ...

movie_names([8, 10, 12, 25, 91])

0        Edison Kinetoscopic Record of a Sneeze (1894)
1                  La sortie des usines Lumière (1895)
2                        The Arrival of a Train (1896)
3    The Oxford and Cambridge University Boat Race ...
4                           Le manoir du diable (1896)
Name: movie, dtype: object

In [29]:
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    closest_neighbors = find_closest_neighbors(user)
    movies_already_seen = movies_watched(user)
    recs = np.array([])
    
    for neighbor in closest_neighbors:
        movies_liked_by_neighbor = movies_liked(neighbor, min_rating=8)
        
        # Add the neighbor's movies, except those already seen by the user
        neighbor_recs = np.setdiff1d(movies_liked_by_neighbor, movies_already_seen, assume_unique=True)
        recs = np.unique(np.concatenate([neighbor_recs, recs], axis=0))
        
        if len(recs) >= num_recs:
            recs = recs[:num_recs]
            break
    
    recommendations = list(movie_names(recs))
    
    return recommendations

In [30]:
# ... and checking ...

make_recommendations(66, num_recs=10)

['Once Upon a Time in America (1984)',
 'Donnie Brasco (1997)',
 'Godzilla (2014)',
 'The Lego Movie (2014)',
 'Despicable Me 2 (2013)',
 'Captain America: The Winter Soldier (2014)',
 'Trance (2013)',
 'Guardians of the Galaxy (2014)',
 'Dawn of the Planet of the Apes (2014)',
 'Muppets Most Wanted (2014)']

In [31]:
def all_recommendations(num_recs=10):
    '''
    INPUT 
        num_recs (int) the (max) number of recommendations for each user
    OUTPUT
        all_recs - a dictionary where each key is a user_id and the value is an array of recommended movie titles
    '''
    # Make the recommendations for each user
    all_users = df_dists['user1'].unique()
    all_recs = dict()
    pbar = ProgressBar()    # the code below runs forever!
    
    for user in pbar(all_users):
        all_recs[user] = make_recommendations(user, num_recs)
    
    return all_recs

all_recs = all_recommendations(10)

100% |########################################################################|


In [32]:
print('all_recs has {} elements\n\n'.format(len(all_recs)))
for user, movie in list(all_recs.items())[:5]:
    print('user {} movies:\n{}\n\n'.format(user, movie))

all_recs has 23512 elements


user 2 movies:
['His Majesty, the Scarecrow of Oz (1914)', 'Three Ages (1923)', 'The Navigator (1924)', 'Sherlock Jr. (1924)', 'The Thief of Bagdad (1924)', 'The Cocoanuts (1929)', 'Frau im Mond (1929)', 'The Green Hornet (1940)', 'The Mark of Zorro (1940)', 'Lifeboat (1944)']


user 3 movies:
['Basic Instinct (1992)', 'The Shawshank Redemption (1994)', 'The Hangover (2009)', 'Limitless (2011)', 'Inception (2010)', "The King's Speech (2010)", 'Love, Rosie (2014)', 'The Intouchables (2011)', 'American Hustle (2013)', 'Divergent (2014)']


user 7 movies:
['The Philadelphia Story (1940)', 'The Magnificent Ambersons (1942)', 'The Third Man (1949)', 'From Here to Eternity (1953)', 'Les vacances de Monsieur Hulot (1953)', 'Shichinin no samurai (1954)', 'The Night of the Hunter (1955)', 'The Searchers (1956)', 'Touch of Evil (1958)', 'The Magnificent Seven (1960)']


user 8 movies:
['Psycho (1960)', 'Jesus Christ Superstar (1973)', 'Monty Python and the Holy Grai

In [33]:
# This loads our solution dictionary so you can compare results
all_recs_sol = pd.read_pickle("data/Term2/recommendations/lesson1/data/all_recs.p")
print('all_recs_sol has {} elements\n\n'.format(len(all_recs_sol)))
for user, movie in list(all_recs_sol.items())[:5]:
    print('user {} movies:\n{}\n\n'.format(user, movie))

all_recs_sol has 23512 elements


user 2 movies:
['Philadelphia (1993)', 'Training Day (2001)', 'About Schmidt (2002)', 'Insomnia (2002)', 'The United States of Leland (2003)', 'Shattered Glass (2003)', 'Man on Fire (2004)', 'Flipped (2010)', 'Silver Linings Playbook (2012)', 'Lawless (2012)', '50/50 (2011)', 'Crazy, Stupid, Love. (2011)', 'The Perks of Being a Wallflower (2012)', 'Before I Go to Sleep (2014)', 'Zero Dark Thirty (2012)', 'American Hustle (2013)', 'Django Unchained (2012)', 'Side Effects (2013)', 'Gone Girl (2014)', 'Enough Said (2013)', 'Nightcrawler (2014)']


user 3 movies:
['Basic Instinct (1992)', 'The Shawshank Redemption (1994)', 'The Wedding Ringer (2015)', 'Life as We Know It (2010)', 'The Hangover (2009)', 'Limitless (2011)', 'The Great Gatsby (2013)', 'This Is Where I Leave You (2014)', 'Inception (2010)', "The King's Speech (2010)", 'Love, Rosie (2014)', 'Now You See Me (2013)', 'The Intouchables (2011)', 'Safe Haven (2013)', 'Before I Go to Sleep (2014)', '

In [34]:
assert all_recs[2] == make_recommendations(2), \
       "Oops!  Your recommendations for user 2 didn't match ours."
assert all_recs[26] == make_recommendations(26), \
       "Oops!  It actually wasn't possible to make any recommendations for user 26."
assert all_recs[1503] == make_recommendations(1503), \
       "Oops! Looks like your solution for user 1503 didn't match ours."

print("If you made it here, you now have recommendations for many users using collaborative filtering!")
HTML('<img src="images/greatjob.webp">')

If you made it here, you now have recommendations for many users using collaborative filtering!


### Now What?

If you made it this far, you have successfully implemented a solution to making recommendations using collaborative filtering. 

`8.` Let's do a quick recap of the steps taken to obtain recommendations using collaborative filtering.  

In [35]:
# Check your understanding of the results by correctly filling in the dictionary below
a = "pearson's correlation and spearman's correlation"
b = 'item based collaborative filtering'
c = "there were too many ratings to get a stable metric"
d = 'user based collaborative filtering'
e = "euclidean distance and pearson's correlation coefficient"
f = "manhattan distance and euclidean distance"
g = "spearman's correlation and euclidean distance"
h = "the spread in some ratings was zero"
i = 'content based recommendation'

sol_dict = {
    'The type of recommendation system implemented here was a ...': d,
    'The two methods used to estimate user similarity were: ': e,
    'There was an issue with using the correlation coefficient.  What was it?': h
}

t.test_recs(sol_dict)

"That's right! All of your solutions look good!"

Additionally, let's take a closer look at some of the results.  There are two solution files that you read in to check your results, and you created these objects

* **df_dists** - a dataframe of user1, user2, euclidean distance between the two users
* **all_recs_sol** - a dictionary of all recommendations (key = user, value = list of recommendations)  

`9.` Use these two objects along with the cells below to correctly fill in the dictionary below and complete this notebook!

In [36]:
a = 567
b = 1503
c = 1319
d = 1325
e = 2526710    # cf. § "Why the NaN's?" above
f = 0
g = 'Use another method to make recommendations - content based, knowledge based, or model based collaborative filtering'

sol_dict2 = {
    'For how many pairs of users were we not able to obtain a measure of similarity using correlation?': e,
    'For how many pairs of users were we not able to obtain a measure of similarity using euclidean distance?': f,
    'For how many users were we unable to make any recommendations for using collaborative filtering?': c,
    'For how many users were we unable to make 10 recommendations for using collaborative filtering?': d,
    'What might be a way for us to get 10 recommendations for every user?': g
}

t.test_recs2(sol_dict2)

"That's right! All of your solutions look good!"

In [37]:
# Use the cells below for any work you need to do!

In [38]:
# NaN euclidean distance values
df_dists['eucl_dist'].isnull().sum()

0

In [40]:
# Users without recs and users with less than 10 recs
num_no_recs = 0
num_less_10_recs = 0

for user, movie_recs in all_recs.items():
    if len(movie_recs) == 0:
        num_no_recs += 1
    elif len(movie_recs) < 10:
        num_less_10_recs +=1

print('No recommendations: ', num_no_recs)    # Found the same value with instructor's code, but not 1319...
print('Less than 10 recommendations: ', num_less_10_recs)    # Unlike instructor's solution, I don't count 0

No recommendations:  1294
Less than 10 recommendations:  17
